In [1]:
import asyncio
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px

import warnings
warnings.filterwarnings("ignore")


#df_c4z = pd.read_csv("order_test_fet_15_11_c4z.csv")
df_26nf = pd.read_csv("csv/test.csv")
#data_c4z=data_c4z[:1000]
#data_26nf=data_26nf[:1000]

#df_index_price = pd.read_csv("2024-11-14.csv")
#df_index_price=df_index_price[:1000]


In [2]:
# #df_c4z.drop(columns=['marketId', 'subaccountId', 'executionType', 'orderHash','quantity'], inplace=True)
df_26nf.drop(columns=['marketId', 'subaccountId', 'executionType', 'orderHash'], inplace=True)


#df_26nf['price'] = df_26nf['price'] * 10**12
#df_c4z['price'] = df_c4z['price'] * 10**12

In [3]:
df_26nf

,orderType,price,quantity,updated_at,state,created_at
0,sell,15.547,480000000,2025-01-22 17:33:33.326,canceled,2025-01-22 17:33:32.674
1,sell,15.553,480000000,2025-01-22 17:33:33.326,booked,2025-01-22 17:33:33.326
2,sell,15.554,480000000,2025-01-22 17:33:32.674,canceled,2025-01-22 17:33:30.133
3,buy,15.539,480000000,2025-01-22 17:33:31.968,canceled,2025-01-22 17:33:31.320
4,buy,15.531,480000000,2025-01-22 17:33:31.968,booked,2025-01-22 17:33:31.968
...,...,...,...,...,...,...
2002,buy,15.531,480000000,2025-01-22 16:34:32.257,canceled,2025-01-22 16:34:27.243
2003,sell,15.548,480000000,2025-01-22 16:34:32.257,canceled,2025-01-22 16:34:31.418
2004,sell,15.553,480000000,2025-01-22 16:34:31.418,canceled,2025-01-22 16:34:30.727
2005,sell,15.548,480000000,2025-01-22 16:34:29.250,canceled,2025-01-22 16:34:27.243


In [4]:
df_transformed = df_26nf.pivot_table(
    index=["updated_at", "state", "created_at"],
    columns="orderType",
    values="price",
    aggfunc="first"
).reset_index()

print(df_transformed.columns) 
# Rename columns for clarity
#df_transformed.columns = ["updated_at", "state", "created_at", "buy_price", "sell_price"]
df_transformed["buy"]= df_transformed["buy"]/1e6
df_transformed["sell"]= df_transformed["sell"]/1e6
#df_transformed["buy_po"]= df_transformed["buy_po"]/1e6
#df_transformed["sell_po"]= df_transformed["sell_po"]/1e6
# Display the transformed DataFrame
df_transformed["offset"] = 100 * (df_transformed["sell"]-df_transformed["buy"])/ ((df_transformed["sell"]+df_transformed["buy"])/2)/2
df_transformed

Index(['updated_at', 'state', 'created_at', 'buy', 'sell'], dtype='object', name='orderType')


orderType,updated_at,state,created_at,buy,sell,offset
0,2025-01-22 16:34:27.243,canceled,2025-01-22 16:34:26.764,NaN,0.000016,NaN
1,2025-01-22 16:34:29.250,canceled,2025-01-22 16:34:27.243,NaN,0.000016,NaN
2,2025-01-22 16:34:31.418,canceled,2025-01-22 16:34:30.727,NaN,0.000016,NaN
3,2025-01-22 16:34:32.257,canceled,2025-01-22 16:34:27.243,0.000016,NaN,NaN
4,2025-01-22 16:34:32.257,canceled,2025-01-22 16:34:31.418,NaN,0.000016,NaN
...,...,...,...,...,...,...
1751,2025-01-22 17:33:31.968,booked,2025-01-22 17:33:31.968,0.000016,NaN,NaN
1752,2025-01-22 17:33:31.968,canceled,2025-01-22 17:33:31.320,0.000016,NaN,NaN
1753,2025-01-22 17:33:32.674,canceled,2025-01-22 17:33:30.133,NaN,0.000016,NaN
1754,2025-01-22 17:33:33.326,booked,2025-01-22 17:33:33.326,NaN,0.000016,NaN


In [5]:
import plotly.graph_objects as go


fig = go.Figure()

# Add traces for each y-column
fig.add_trace(go.Scatter(x=df_transformed['updated_at'], y=df_transformed['buy'],
                         mode='lines', name='buy '))
fig.add_trace(go.Scatter(x=df_transformed['updated_at'], y=df_transformed['sell'],
                         mode='lines', name='sell '))
#fig.add_trace(go.Scatter(x=df_transformed['updated_at'], y=df_transformed['buy_po'],
#                         mode='lines', name='buy '))
#fig.add_trace(go.Scatter(x=df_transformed['updated_at'], y=df_transformed['sell_po'],
#                         mode='lines', name='sell '))
# Update layout for titles and axes
fig.update_layout(
    title="Offsets over Time",
    xaxis_title="Updated At",
    yaxis_title="Offset Value",
    legend_title="Offsets",
)

# Show the figure
fig.show()


In [6]:
import plotly.express as px

# Create a sample DataFrame

# Create a line plot from the DataFrame
fig = px.line(df_transformed, x='updated_at', y='offset', title='Offset from c4z order')

# Show the plot
fig.show()
